# Отчет о создании и обучении на датасете Fashion MNIST модели CNN (гипотеза 6, реализация 1)

## Содержание

- Список терминов, специфичных для темы отчета
- Введение
- 1 Создание и обучение модели
 - 1.1 Загрузка датасета
 - 1.2 Создание модели
 - 1.3 Обучение модели
 - 1.4 Сохранение модели на диск
- 2 Рекомендации по дальнейшему улучшению/развитию модели
- Ссылки

## Список терминов, специфичных для темы отчета

См. раздел "Машинное обучение" документа "Словарь терминов, специфичных для выполняемых работ и изучаемых материалов в период с 25 сентября 2020 г. по 25 ноября 2020 г.".

## Введение

В настоящем отчете приведен код, соответствующий созданию и обучению модели CNN на датасете Fashion MNIST, а также рекомендации по дальнейшему улучшению/развитию модели. Используемый язык программирования – Python. Для создания и обучения модели используются библиотеки MXNet и Gluon.

## 1 Создание и обучение модели

Импортирование необходимых модулей и функций:

In [1]:
import mxnet as mx
from mxnet.optimizer import Adam
from mxnet import gluon, autograd
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms

Установка вычислительного контекста (для использования при обучении видеокарты, если это возможно):

In [2]:
ctx = [mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()]
print('context:', ctx)

context: [gpu(0)]


### 1.1 Загрузка датасета

Загрузка датасета и установка размера батча равным 128:

In [3]:
batch_size = 128

mnist_train = datasets.FashionMNIST(train=True)
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.13, 0.31)])
mnist_train = mnist_train.transform_first(transformer)
train_data = gluon.data.DataLoader(
    mnist_train, batch_size=batch_size, shuffle=True, num_workers=4)

mnist_valid = gluon.data.vision.FashionMNIST(train=False)
val_data = gluon.data.DataLoader(
    mnist_valid.transform_first(transformer),
    batch_size=batch_size, num_workers=4)

### 1.2 Создание модели

Создание модели, состоящей из 4 слоев:
1. Сверточный слой с параметрами:
	- Количество каналов: 8
	- Размер ядра свертки: 4х4
	- Функция активации: гиперболический тангенс
2. Слой пулинга с параметрами:
	- Размер ядра пулинга: 2х2
	- Шаг: 2
3. Полносвязный слой с параметрами:
	- Количество нейронов: 16
	- Функция активации: гиперболический тангенс
4. Полносвязный слой с параметрами:
	- Количество нейронов: 10
	- Функция активации: гиперболический тангенс

In [4]:
net = nn.HybridSequential()
with net.name_scope():
    net.add(nn.Conv2D   (channels=8, kernel_size=4, activation='tanh'))
    net.add(nn.MaxPool2D(pool_size=2, strides=2))
    net.add(nn.Dense    (16, activation='tanh'))
    net.add(nn.Dense    (10, activation='tanh'))
net.hybridize()

### 1.3 Обучение модели

Для инициализации сети используется метод Завьера (Xavier). В качестве функции потерь используется SCE (Softmax Cross-Entropy loss). В качестве оптимизатора используется Adam (Adaptive Moment Estimation – метод Адаптивной Оценки Моментов). Обучение производится на протижение 100 эпох.

In [5]:
epochsForTraining = 150
#                   ^^^--- 100 -> 150
validationAccToTrain = 0.90

def test(ctx):
    metric = mx.metric.Accuracy()
    for data, label in val_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        metric.update([label], [output])

    return metric.get()

def train(net, ctx):
    # Collect all parameters from net and its children, then initialize them.
    net.initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
    # Trainer is for updating parameters with gradient.
    trainer = gluon.Trainer(net.collect_params(), Adam())
    metric = mx.metric.Accuracy()
    loss = gluon.loss.SoftmaxCrossEntropyLoss()

    maxValidationAcc = 0
    for i in range(epochsForTraining):
        # reset data iterator and metric at begining of epoch.
        metric.reset()
        for j, (data, label) in enumerate(train_data):
            # Copy data to ctx if necessary
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            # Start recording computation graph with record() section.
            # Recorded graphs can then be differentiated with backward.
            with autograd.record():
                output = net(data)
                L = loss(output, label)
                L.backward()
            # take a gradient step with batch_size equal to data.shape[0]
            trainer.step(data.shape[0])
            # update metric at last.
            metric.update([label], [output])

        name, acc = metric.get()
        name, validationAcc = test(ctx)
        epoch = i + 1
        print('epoch', ('{:2.0f}:').format(epoch), '\t', "{:.2f}%".format(acc * 100), '- train set accuracy'
              '\n\t\t', "{:.2f}%".format(validationAcc * 100), '- validation set accuracy')
        if validationAcc > maxValidationAcc:
            maxValidationAcc = validationAcc
            if validationAcc >= validationAccToTrain:
                net.export('exported/150/fashion_mnist', epoch=epoch)
                print('saved epoch', epoch)
        print()
    print('Best result: {:.2f}%'.format(maxValidationAcc * 100))

In [6]:
train(net, ctx[0])

epoch  1: 	 77.08% - train set accuracy
		 82.60% - validation set accuracy

epoch  2: 	 83.91% - train set accuracy
		 85.34% - validation set accuracy

epoch  3: 	 85.86% - train set accuracy
		 86.31% - validation set accuracy

epoch  4: 	 86.92% - train set accuracy
		 87.35% - validation set accuracy

epoch  5: 	 87.41% - train set accuracy
		 86.98% - validation set accuracy

epoch  6: 	 87.83% - train set accuracy
		 87.62% - validation set accuracy

epoch  7: 	 88.20% - train set accuracy
		 88.44% - validation set accuracy

epoch  8: 	 88.34% - train set accuracy
		 88.58% - validation set accuracy

epoch  9: 	 88.69% - train set accuracy
		 87.83% - validation set accuracy

epoch 10: 	 88.96% - train set accuracy
		 88.72% - validation set accuracy

epoch 11: 	 89.18% - train set accuracy
		 88.56% - validation set accuracy

epoch 12: 	 89.27% - train set accuracy
		 88.60% - validation set accuracy

epoch 13: 	 89.51% - train set accuracy
		 88.39% - validation set accuracy


Максимальная достигнутая точность на проверочной части датасета составляет **89.82%**

### 1.4 Сохранение модели на диск

Обучение модели выполняется посредством функции `train` (см. раздел 1.3). Функция `train` использует глобальные переменные `validationAccToTrain` и `epochsForTraining`. Значение глобальной переменной `validationAccToTrain` соответствует точности (accuracy) на проверочной части датасета, при достижении которой и одновременно максимальной точности в ходе обучения, необходимо сохранить архитектуру и параметры модели на диск. Сохранение модели на диск выполняется посредством функции `net.export`, при этом создаются/перезаписываются два файла: **fashion-mnist-symbol.json**, содержащий информацию, определяющую архитектуру модели, и **fashion-mnist-<номер эпохи>.params**, содержащий параметры модели, полученные в ходе обучения и соответствующие эпохе <номер эпохи>. Значение глобальной переменной `epochsForTraining` соответствует количеству эпох, в течение которых необходимо проводить обучение.

## Ссылки

1. Официальный сайт mxnet: https://mxnet.apache.org/